In [44]:
import torch.optim as optim
import torch
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import random
import os
%run helpers_preproc.ipynb
%run alternate_data.ipynb

In [45]:
origDataDir = 'SHREC11/'
newDataDir = 'SHREC11_plus/'

In [46]:
#generate additional data and update label
origLabel = np.array(readLbl(600,origDataDir + 'labels.txt'),dtype = int) #first take in the original labels
newLabelFile = open(newDataDir + "labels.txt", "w")

In [47]:
radiuss = [0.0002,0.0004,0.0008,0.0016]

for i in range(600):
    #First create the new obj file
    origN = origDataDir + 'T' + str(i) + '.obj'
    
    for j in range(len(radiuss)):
        radius = radiuss[j]
        newN = newDataDir + 'T' + str(i+600*(j)) + '.obj'
    
        #newLabelFile.write("T" + str(i) + ".obj" + " " + str(origLabel[i]) + "\n")
        if not (write_new_files(origN,newN,radius)):
            print('Data generation failed on ' , i)
        
        newLabelFile.write("T" + str(i + 600*(j)) + ".obj" + " " + str(origLabel[i]) + "\n")

    
newLabelFile.close()

In [48]:
#read in the data, get normalized adjacency (NxN)
num_meshes = 600 * 4
train_size = 450 * 4
test_size = 150 * 4
mesh_dir = 'SHREC11_plus/'

In [49]:
num_swaps = 10000

labels = np.array(readLbl(num_meshes,mesh_dir+'labels.txt'))

for swap_idx in range(num_swaps):
    idxA = random.randint(0,num_meshes-1)
    idxB = random.randint(0,num_meshes-1)
    
    if idxA != idxB:
        labelA = labels[idxA]
        labelB = labels[idxB]
        labels[idxA] = labelB
        labels[idxB] = labelA

        #change filenames
        os.rename(mesh_dir+'T' + str(idxA) + '.obj',mesh_dir+'temp.obj')
        os.rename(mesh_dir+'T' + str(idxB) + '.obj',mesh_dir+'T' + str(idxA) + '.obj')
        os.rename(mesh_dir+'temp.obj',mesh_dir+'T' + str(idxB) + '.obj')

newLabelFile = open(newDataDir + "labels.txt", "w")
for i in range(num_meshes):
        newLabelFile.write("T" + str(i) + ".obj" + " " + str(labels[i]) + "\n")


newLabelFile.close()

In [50]:
label_np = np.array(readLbl(num_meshes,mesh_dir+'labels.txt'))

In [51]:
node_sigs_list = []
normed_adjMats_list = []

for i in range(num_meshes):
    fName = 'T' + str(i) + '.obj'
    adj_noscale_read = obj_2_adj_noscale(mesh_dir + fName)
    adj_scaled_read = obj_2_adj(mesh_dir + fName)

    if (adj_scaled_read.shape[0] < 252):
        adj_noscale = np.empty([252,252])
        adj_scaled = np.empty([252,252])
    else:
        adj_noscale = adj_noscale_read
        adj_scaled = adj_scaled_read

    adj_normalized = adj_noscale / np.reshape(np.sum(adj_noscale,axis = 1),[adj_noscale.shape[0],1])

    #node level signal extraction
    node_degs = np.sum(adj_noscale,axis = 0)
    node_neigh_max = np.max(adj_scaled,axis = 0)
    node_neigh_min = np.min(adj_scaled,axis = 0)
    node_neigh_sum = np.sum(adj_scaled,axis = 0)
    node_neigh_mean = np.sum(adj_scaled,axis = 0)

    node_sig = np.stack([node_degs,node_neigh_max,node_neigh_min,node_neigh_sum,node_neigh_mean],axis = 1)
    
    node_sigs_list.append(node_sig)
    normed_adjMats_list.append(adj_normalized)

In [52]:
normed_adjMats = np.nan_to_num(np.stack(normed_adjMats_list))
node_sigs = np.nan_to_num(np.stack(node_sigs_list))

In [53]:
#feature_mean = np.reshape(np.mean(node_sigs,axis = 1),[num_meshes,1,5])
#node_sigs = node_sigs - feature_mean 

In [54]:
node_sigs

array([[[4.        , 0.13550079, 0.        , 0.42369974, 0.42369974],
        [7.        , 0.17212504, 0.        , 0.98396842, 0.98396842],
        [5.        , 0.16103479, 0.        , 0.68239826, 0.68239826],
        ...,
        [7.        , 0.19047461, 0.        , 1.03865331, 1.03865331],
        [4.        , 0.16742295, 0.        , 0.53104589, 0.53104589],
        [5.        , 0.294856  , 0.        , 0.85576911, 0.85576911]],

       [[4.        , 0.17414296, 0.        , 0.43763473, 0.43763473],
        [5.        , 0.15670403, 0.        , 0.62843306, 0.62843306],
        [5.        , 0.17932172, 0.        , 0.68915266, 0.68915266],
        ...,
        [7.        , 0.25971837, 0.        , 1.00513952, 1.00513952],
        [6.        , 0.14572146, 0.        , 0.55284372, 0.55284372],
        [7.        , 0.1736732 , 0.        , 0.95802574, 0.95802574]],

       [[5.        , 0.08813521, 0.        , 0.3568331 , 0.3568331 ],
        [7.        , 0.10132539, 0.        , 0.46379741, 0.4

In [55]:
normed_adjMats_tr = normed_adjMats[:train_size,:,:]
node_sigs_tr = node_sigs[:train_size,:,:]
label_np_tr = label_np[:train_size]

normed_adjMats_ts = normed_adjMats[train_size:,:,:]
node_sigs_ts = node_sigs[train_size:,:,:]
label_np_ts = label_np[train_size:]

label_mat_tr = torch.tensor(np.where(igl.all_pairs_distances(label_np_tr,label_np_tr,False) > 0.5,0,1)).float()
label_mat_ts = torch.tensor(np.where(igl.all_pairs_distances(label_np_ts,label_np_ts,False) > 0.5,0,1)).float()

In [56]:
normed_adjMats_tr = torch.tensor(normed_adjMats_tr).float()
node_sigs_tr = torch.tensor(node_sigs_tr).float()
label_tr = torch.tensor(label_np_tr).float()

normed_adjMats_ts = torch.tensor(normed_adjMats_ts).float()
node_sigs_ts = torch.tensor(node_sigs_ts).float()
label_ts = torch.tensor(label_np_ts).float()

In [57]:
#label_mat_train = form_label_matrix(label_mat, train_dict)

In [58]:
#normed_adjMats_list_train, node_sigs_list_train = prep_data(train_dict)

In [59]:
#normed_adjMats_train = np.stack(normed_adjMats_list_train)
#node_sigs_train = np.stack(node_sigs_list_train)

#normed_adjMats_train = torch.tensor(normed_adjMats_train,requires_grad=False).float()
#node_sigs_train = torch.tensor(node_sigs_train,requires_grad=False).float()

#normed_adjMats_train = torch.nan_to_num(normed_adjMats_train,0,0,0)
#node_sigs_train = torch.nan_to_num(node_sigs_train,0,0,0)

In [60]:
node_sigs_tr.shape

torch.Size([1800, 252, 5])

In [61]:
%run NNs.ipynb
g = GCN(5,[5,5,5,5])
n = MLP(252*5,[5,5,5,5])
output1 = g.forward(normed_adjMats_tr,node_sigs_tr)
output2 = n.forward(output1)
print(output1.shape)
print(output2.shape)

torch.Size([1800, 252, 5])
torch.Size([1800, 5])


In [62]:

def lossF(features,lbl_mat):
    disMat = torch.cdist(features,features)
    sameType = disMat * (lbl_mat-torch.diag(torch.ones(lbl_mat.shape[0])))
    diffType = disMat * (1-lbl_mat)

    diffTypeScaled = diffType * torch.exp(-diffType)
    diffTypeScaledMean = torch.sum(diffTypeScaled)/ torch.count_nonzero(diffTypeScaled)
    diffTypeMean = torch.sum(diffType)/ torch.count_nonzero(diffType)

    sameTypeMean = torch.sum(sameType) / torch.count_nonzero(sameType)
    sameTypeStd = torch.sum((sameType - sameTypeMean)**2) / torch.count_nonzero(sameType)

    #toRet = -(sameTypeMean-diffTypeMean)**2 # + 0.01 * torch.sqrt(sameTypeStd)
    #toRet = -(sameTypeMean-diffTypeMean)**2 #+ 0.5*(sameTypeMean + 0.01 * torch.sqrt(sameTypeStd))
    toRet = -(sameTypeMean-diffTypeMean)**2 - diffTypeScaledMean + 0.5 * torch.sqrt(sameTypeStd)
    
    print(sameTypeMean.detach().numpy(),diffTypeMean.detach().numpy(),torch.sqrt(sameTypeStd).detach().numpy(), toRet.detach().numpy())
    return  toRet

In [64]:
optimizer = optim.Adam(g.weights + n.weights,lr = 0.0001)
for i in range(100000):
    optimizer.zero_grad()
    output = n.forward(g.forward(normed_adjMats_tr,node_sigs_tr))
    loss = lossF(output,label_mat_tr)
    loss.backward()
    optimizer.step()

0.009815609 0.011271889 0.05397419 0.015909836
0.009717828 0.011211275 0.05336741 0.015658468
0.009472763 0.011008783 0.052121464 0.015239498
0.00926116 0.010752921 0.05092417 0.014887024
0.009374454 0.010783215 0.0519943 0.015427266
0.009049778 0.010476916 0.04978067 0.014581689
0.009061721 0.010508302 0.049872242 0.014601457
0.008726695 0.010164029 0.04800009 0.013994484
0.008963561 0.010384583 0.049749646 0.01469036
0.008635034 0.010129281 0.047522508 0.013792751
0.0087919375 0.010210031 0.048792332 0.014378063
0.008653564 0.010141754 0.04764602 0.013844778
0.008573455 0.009995102 0.047163233 0.013741373
0.00846559 0.0099236695 0.04686252 0.013680123
0.008201751 0.009648194 0.045134146 0.013063062
0.008094112 0.009483714 0.04454387 0.012926587
0.0080969 0.00954114 0.044558536 0.012879364
0.00816806 0.009536965 0.04517312 0.013206965
0.007848002 0.009199285 0.043153428 0.012506372
0.007963789 0.009263649 0.04385141 0.012797758
0.007758826 0.009096805 0.042682566 0.012371682
0.0077661

KeyboardInterrupt: 

In [ ]:
output = n.forward(g.forward(normed_adjMats_tr,node_sigs_tr))
output.shape

In [ ]:
disMat = torch.cdist(output,output).flatten()
mask_same = torch.tensor(label_mat_tr - torch.diag(torch.ones(train_size)),dtype=bool).flatten()
mask_diff = torch.tensor(1 - label_mat_tr,dtype=bool).flatten()
sameComp = disMat[mask_same].detach().numpy()
diffComp = disMat[mask_diff].detach().numpy()
weightsSame = np.ones_like(sameComp) / len(sameComp)
weightsDiff = np.ones_like(diffComp) / len(diffComp)

In [ ]:
print(np.mean(sameComp),np.mean(diffComp))
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(sameComp,bins=np.arange(0,np.max(sameComp),0.01),weights=weightsSame,log = False)
ax1.set_title('Same Comp')
ax2.hist(diffComp,bins=np.arange(0,np.max(diffComp),0.01),weights=weightsDiff,log = False)
ax2.set_title('Diff Comp')

In [ ]:
output = n.forward(g.forward(normed_adjMats_ts,node_sigs_ts))
output.shape

In [ ]:
disMat = torch.cdist(output,output).flatten()
mask_same = torch.tensor(label_mat_ts - torch.diag(torch.ones(test_size)),dtype=bool).flatten()
mask_diff = torch.tensor(1 - label_mat_ts,dtype=bool).flatten()
sameComp = disMat[mask_same].detach().numpy()
diffComp = disMat[mask_diff].detach().numpy()
weightsSame = np.ones_like(sameComp) / len(sameComp)
weightsDiff = np.ones_like(diffComp) / len(diffComp)

In [ ]:
print(np.mean(sameComp),np.mean(diffComp))
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(sameComp,bins=np.arange(0,np.max(sameComp),0.01),weights=weightsSame,log = False)
ax1.set_title('Same Comp')
ax2.hist(diffComp,bins=np.arange(0,np.max(diffComp),0.01),weights=weightsDiff,log = False)
ax2.set_title('Diff Comp')

In [ ]:
#from matplotlib.pyplot import figure
#figure(figsize=(10, 8), dpi=80)


In [ ]:
BigDiffIndices = np.where(disMat.detach().numpy().flatten() > 0.75)[0].tolist()

In [ ]:
len(BigDiffIndices)

In [ ]:
vizMat = np.zeros([30,30])

In [ ]:
#Separating 30 categories into less categories

# for index in BigDiffIndices:
#     row, col = divmod(index,test_size)
#     cat1, cat2 = int(label_np_ts[row]),int(label_np_ts[col])
#     if cat1 == cat2:
#         print(cat1, cat2)
#     vizMat[cat1,cat2] += 1
#     vizMat[cat2,cat1] += 1


In [ ]:
vizMat = vizMat / np.max(vizMat)

In [ ]:
plt.matshow(vizMat)

In [ ]:
features = output.detach().numpy()

In [ ]:
features.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)
newX = pca.fit_transform(features)
np.sum(pca.explained_variance_ratio_)

In [ ]:
newX.shape

In [ ]:
indices_to_show = [0,1,2,3,4]

In [ ]:
dataPlot = []
for i in range(len(indices_to_show)):
    a = newX[np.where(label_np_ts == indices_to_show[i]),:]
    a = np.reshape(a,[a.shape[1],a.shape[2]])
    categories = np.ones([a.shape[0],1]) * indices_to_show[i]
    b = np.concatenate((a,categories),axis = 1)
    dataPlot.append(b)
    
dataPlot = np.concatenate(dataPlot,axis = 0)

In [ ]:
dataPlot.shape

In [ ]:
plt.clf()

In [ ]:
from mpl_toolkits.mplot3d import axes3d

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(dataPlot[:,0], dataPlot[:,1], dataPlot[:,2], label=dataPlot[:,3], c = dataPlot[:,3], cmap = 'viridis')
#ax.axis('off')
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
ax.axes.zaxis.set_ticklabels([])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('[0,1,2,3,4]')

#plt.legend(loc="upper right")

plt.show()